In [ ]:
# Now we will use blastx and uniprot apis to get relevant descriptions, keywords, make ori-flag column 
# and Extract the nucleotide region from the start to end of the entire rep origin

In [ ]:
!pip install biopython --upgrade

In [1]:
import numpy as np
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from IPython.display import clear_output

### First we will appraoch to work with a small subset of data for performance related issues 

In [2]:
full_df = pd.read_csv("/scratch/alopatki_lab/Sharma/summer_project/db_wo_blast_uniprot.csv")

# sample_df = full_df.head(int(full_df.shape[0]*0.1))

In [9]:
import pickle

# Load the pickle file
with open('/scratch/alopatki_lab/Sharma/summer_project/temp_files/cds_dict_cleaned_v1.pkl', 'rb') as f:
    all_cds = pickle.load(f)

# Now you can work with the loaded data
print(len(all_cds.keys()))


276


In [42]:
print(list(all_cds.keys())[0])
print(all_cds[list(all_cds.keys())[0]])

rop
['...']


In [5]:
# Defing functions we will use 

def extract_cds(row):
    if row["Feature Type"] == "CDS":
        if row["Feature Label"] in all_cds.keys():
            return row["Feature Sequence"]
        else:
            return -1
    else:
        return -1

def blast_search(cds_seq):
    result_handle = NCBIWWW.qblast("blastx", "nr",cds_seq)
    
    record = NCBIXML.read(result_handle)

    result_handle.close()

    # We will get the top 10 results, their title and sequences
    titles = []
    sequences = []
    i = 1
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            if i <= 10:
                sequences.append(hsp.sbjct)
                title = alignment.title
                titles.append(title)
                i += 1
    return [titles, sequences]

  


In [47]:
# Maintaining a dictionary of cds features and their blast results so that we do not have to do it repeatedly
cds_blasts_done = {}

# Adding new column for blast results
full_df["Blast titles"] = None
full_df["Blast Prot Seq"] = None
# Doing the blastx search row by row only for cds feature labels that are in all_cds dict

for i in range(full_df.shape[0]):
    print(f"Currently Processing row: {i+1}/{full_df.shape[0]}")
    curr_row = full_df.iloc[i]
    # print(curr_row)
    seq = extract_cds(curr_row)
    # print(seq)
    if seq == -1:
        clear_output(wait = True)
        continue
    else:
        feature_label = curr_row["Feature Label"]
        # print(feature_label)
        if feature_label in cds_blasts_done.keys():
            full_df.at[i, "Blast titles"] = cds_blasts_done[feature_label][0]
            full_df.at[i, "Blast Prot Seq"] = cds_blasts_done[feature_label][1]
        else:
            result = blast_search(seq)
            cds_blasts_done[feature_label] = result
            full_df.at[i, "Blast titles"] = cds_blasts_done[feature_label][0]
            full_df.at[i, "Blast Prot Seq"] = cds_blasts_done[feature_label][1]
        clear_output(wait = True)      
            
        

Currently Processing row: 162/281556
trfA


URLError: <urlopen error [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)>

In [3]:
import pickle
from IPython.display import clear_output
# Load the pickle file
with open("/scratch/alopatki_lab/Sharma/summer_project/temp_files/cds_blast_done.pkl", 'rb') as f:
    cds_blasts_done = pickle.load(f)

In [10]:
from requests.exceptions import SSLError
from time import sleep

MAX_RETRIES = 100 # Adjust as needed
RETRY_DELAY = 3  # Seconds

# Adding new column for blast results
full_df["Blast titles"] = None
full_df["Blast Prot Seq"] = None

for i in range(full_df.shape[0]):
    retry_count = 0

    while retry_count < MAX_RETRIES:
        print(f"Processing row: {i+1}/{full_df.shape[0]} (Retry: {retry_count})")
        curr_row = full_df.iloc[i]

        try:
            seq = extract_cds(curr_row)

            if seq == -1:  
                clear_output(wait = True)
                break  # Skip to the next row if extraction fails
                

            feature_label = curr_row["Feature Label"]
            print("Feature Label:", feature_label)

            if feature_label in cds_blasts_done:  
                full_df.at[i, "Blast titles"] = cds_blasts_done[feature_label][0]
                full_df.at[i, "Blast Prot Seq"] = cds_blasts_done[feature_label][1]
                clear_output(wait = True)

            else:
                result = blast_search(seq) 
                cds_blasts_done[feature_label] = result
                full_df.at[i, "Blast titles"] = result[0]  # Simplified assignment
                full_df.at[i, "Blast Prot Seq"] = result[1]
                clear_output(wait = True)

            break  # Exit retry loop on successful execution

        except Exception as e:
            print(f"Error encountered {e}. Retrying...")
            retry_count += 1
            sleep(RETRY_DELAY)  # Wait before retrying

    if retry_count == MAX_RETRIES:
        print(f"WARNING: Failed to process row {i+1} after {MAX_RETRIES} attempts.")

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
with open("/scratch/alopatki_lab/Sharma/summer_project/temp_files/cds_blast_done.pkl", "wb") as file:
    # Pickle the dictionary and write it to the file
    pickle.dump(cds_blasts_done, file)


# with open("/home/ashar58/Downloads/New temp/cds_blast_done.pkl", "wb") as file:
#     # Pickle the dictionary and write it to the file
#     pickle.dump(cds_blasts_done, file)

In [18]:
# Keeping a reference dataframe as backup
full_df.to_csv("/scratch/alopatki_lab/Sharma/summer_project/db_blast_only.csv")

In [34]:
# In the blast column we will only keep the description of the hits as list
import re

def is_replication_related(text):
  """
  Checks if the given text is related to replication, ignoring text within brackets.

  Args:
    text: The text to check.

  Returns:
    True if the text is related to replication, False otherwise.
  """

  # Remove text within brackets
  text = re.sub(r'\[.*?\]', '', text)

  # Define patterns and keywords for replication-related terms
  patterns = [
      r"\brop\b",  # Match "rop" as a whole word
      r"replication",
      r"plasmid",
      r"copy\s*number",
      r"ori",
      r"rep(?:lication)?\s*(?:protein|factor|initiator)?",
      r"partition",
      r"segregation",
      r"conjugation",
      r"transformation",
      r"transduction",
  ]
  keywords = ["dnaa", "dnab", "dnac", "dna polymerase", "helicase", "primase"]

  # Check for matches with patterns and keywords
  if any(re.search(pattern, text, re.IGNORECASE) for pattern in patterns):
    return True
  if any(keyword in text.lower() for keyword in keywords):
    return True

  return False


In [56]:
import re

def is_replication_related2(text):
  """
  Checks if the given text is related to replication, ignoring text within brackets.
  Improved for higher accuracy and reduced false positives.

  Args:
    text: The text to check.

  Returns:
    True if the text is related to replication, False otherwise.
  """

  # Remove text within brackets
  text = re.sub(r'\[.*?\]', '', text)

  # Define specific patterns for replication-related proteins
  patterns = [
      r"\brop\b",  # Match "rop" as a whole word
      r"replication\s*(?:protein|factor|initiator)?", 
      r"plasmid\s*(?:replication|copy\s*number|partition|segregation)?",
      r"ori\s*(?:region|sequence)?",
      r"dnaa\s*(?:protein)?",
      r"dnab\s*(?:helicase)?",
      r"dnac\s*(?:protein)?",
      r"dna\s*polymerase\s*(?:i|iii)?", 
      r"primase",
      r"replisome",
      r"topoisomerase",
      r"gyrase",
      r"dna\s*binding\s*protein\s*(?:involved\s*in\s*replication)?",
  ]

  # Check for matches with the specific patterns
  if any(re.search(pattern, text, re.IGNORECASE) for pattern in patterns):
    return True

  # Additional check for terms that might indicate replication but require context
  # These terms are more likely to be false positives on their own
  potential_terms = ["conjugation", "transformation", "transduction"]
  for term in potential_terms:
    if re.search(term, text, re.IGNORECASE):
      # Add more specific checks based on your domain knowledge
      # For example, check if the term is near other replication-related words
      if "plasmid" in text.lower() or "dna" in text.lower():
        return True

  return False

In [59]:
# Example usage
exindex = -1
for i in range(len(cds_blasts_done)):
    if list(cds_blasts_done.keys())[i] == 'LshC2c2':
        exindex = i
        break
# print(exindex)

for things in cds_blasts_done[list(cds_blasts_done.keys())[exindex]][0]:
    print(things)
    print()
    # if is_replication_related(things):
    #     print(f"{list(cds_blasts_done.keys())[exindex]} - Replication Related")
    #     break

        

pdb|7DMQ|A Chain A, CRISPR/Cas system Cas13a [Leptotrichia shahii]

ref|WP_018451595.1| type VI-A CRISPR-associated RNA-guided ribonuclease Cas13a [Leptotrichia shahii] >sp|P0DOC6.1| RecName: Full=CRISPR-associated endoribonuclease Cas13a; AltName: Full=CRISPR-associated endoribonuclease C2c2; Short=EndoRNase; AltName: Full=LshC2c2 [Leptotrichia shahii DSM 19757] >dbj|BBM39911.1| CRISPR/Cas system Cas13a [Leptotrichia shahii]

pdb|5WTK|A Chain A, CRISPR-associated endoribonuclease C2c2 [Leptotrichia shahii DSM 19757]

pdb|5WTJ|A Chain A, CRISPR-associated endoribonuclease C2c2 [Leptotrichia shahii DSM 19757] >pdb|5WTJ|B Chain B, CRISPR-associated endoribonuclease C2c2 [Leptotrichia shahii DSM 19757]

ref|WP_304078717.1| type VI-A CRISPR-associated RNA-guided ribonuclease Cas13a [Leptotrichia wadei] >gb|MBS6019491.1| type VI-A CRISPR-associated RNA-guided ribonuclease Cas13a [Leptotrichia wadei]

ref|WP_021744063.1| type VI-A CRISPR-associated RNA-guided ribonuclease Cas13a [Leptotrichi

In [57]:
# # Example usage
# exindex = -1
# for i in range(len(cds_blasts_done)):
#     if list(cds_blasts_done.keys())[i] == 'pBBR1 Rep':
#         exindex = i
#         break
# # print(exindex)
# for i in range(len(cds_blasts_done)):
#     for things in cds_blasts_done[list(cds_blasts_done.keys())[i]][0]:
#         if is_replication_related(things):
#             print(f"{list(cds_blasts_done.keys())[i]} - Replication Related")
            # break
    
# Example usage
exindex = -1
for i in range(len(cds_blasts_done)):
    if list(cds_blasts_done.keys())[i] == 'pBBR1 Rep':
        exindex = i
        break
# print(exindex)
for i in range(len(cds_blasts_done)):
    for things in cds_blasts_done[list(cds_blasts_done.keys())[i]][0]:
        if is_replication_related2(things):
            print(f"{list(cds_blasts_done.keys())[i]} - Replication Related")
            break

rop - Replication Related
pBBR1 Rep - Replication Related
trfA - Replication Related
pVS1 StaA - Replication Related
pVS1 RepA - Replication Related
Rep101 - Replication Related
RSF1010 RepB - Replication Related
RSF1010 RepA - Replication Related
lef2 - Replication Related
TRP1 - Replication Related
pRO1600 Rep - Replication Related
Rep101(Ts) - Replication Related
Cdt1 (30-120) - Replication Related
Gem1 (1-110) - Replication Related
Gam - Replication Related
Mxe GyrA intein - Replication Related
CBD - Replication Related
RSF1010 RepC - Replication Related
traJ - Replication Related
pSG5 Rep - Replication Related
repE - Replication Related
sopA - Replication Related
sopB - Replication Related
repB - Replication Related
Taq polymerase - Replication Related
GB1 - Replication Related
Rta AD - Replication Related
RepA - Replication Related
M13 gene V - Replication Related
M13 gene II - Replication Related
LshC2c2 - Replication Related
ADE2 - Replication Related
Csy4 - Replication Related

In [63]:
full_df['Ori Flag'] = 0

for i in range(full_df.shape[0]):
    curr_row = full_df.iloc[i]
    if curr_row["Feature Type"] == "rep_origin" or curr_row["Feature Type"] == "oriT":
        full_df.at[i, 'Ori Flag'] = 1
    if curr_row["Feature Type"] == "CDS":
        if curr_row["Feature Label"] in cds_blasts_done.keys():
            full_df.at[i, 'Blast titles'] = cds_blasts_done[curr_row["Feature Label"]][0]
            full_df.at[i, 'Blast Prot Seq'] = cds_blasts_done[curr_row["Feature Label"]][1]
            for things in cds_blasts_done[curr_row["Feature Label"]][0]:
                if is_replication_related2(things):
                    full_df.at[i, 'Ori Flag'] = 1

In [65]:
full_df.head(100)

,Plasmid ID,Plasmid Name,Vector backbone,Vector type,Bacterial Resistance(s),Growth Temperature,Growth Strain(s),Copy number,Gene/Insert name,Species,...,Topology,Feature Type,Feature Label,Feature Note,Feature Location,Feature Sequence,Feature Length,Blast titles,Blast Prot Seq,Ori Flag
0,176079,OA-1010B,Addgene #104968,bacterial,"Ampicillin, 100 μg/mL",37°C,DH5alpha,Unknown,L596_g25050.t1,D. melanogaster (fly),...,circular,source,Sequence,Full plasmid Sequence,[0:8989](+),GAGTACTGTCCTCCGAGCGGAGTACTGTCCTCCGAGCGGAGTACTG...,8989,None,None,0
1,176079,OA-1010B,Addgene #104968,bacterial,"Ampicillin, 100 μg/mL",37°C,DH5alpha,Unknown,L596_g25050.t1,D. melanogaster (fly),...,circular,protein_bind,5X UAS,"five tandem copies of the ""ScaI site"" 17-mer C...",[126:221](+),CGGAGTACTGTCCTCCGAGCGGAGTACTGTCCTCCGAGCGGAGTAC...,95,None,None,0
2,176079,OA-1010B,Addgene #104968,bacterial,"Ampicillin, 100 μg/mL",37°C,DH5alpha,Unknown,L596_g25050.t1,D. melanogaster (fly),...,circular,promoter,hsp70 promoter,Drosophila melanogaster hsp70Bb promoter,[251:490](+),CCGGAGTATAAATAGAGGCGCTTCGTCTACGGAGCGACAATTCAAT...,239,None,None,0
3,176079,OA-1010B,Addgene #104968,bacterial,"Ampicillin, 100 μg/mL",37°C,DH5alpha,Unknown,L596_g25050.t1,D. melanogaster (fly),...,circular,primer_bind,pCasper-hs,"Drosophila Hsp70 promoter, forward primer",[436:458](+),GCAACTACTGAAATCTGCCAAG,22,None,None,0
4,176079,OA-1010B,Addgene #104968,bacterial,"Ampicillin, 100 μg/mL",37°C,DH5alpha,Unknown,L596_g25050.t1,D. melanogaster (fly),...,circular,CDS,3xHA,NaN,[1148:1238](+),TACCCCTACGACGTGCCCGACTACGCCGGCTACCCATACGACGTCC...,90,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,112198,pANY6,pAny6,bacterial,"Kanamycin, 50 μg/mL",37°C,ccdB Survival,Unknown,NaN,NaN,...,circular,primer_bind,pBR322ori-F,"pBR322 origin, forward primer",[3918:3938](+),GGGAAACGCCTGGTATCTTT,20,None,None,0
96,170078,PclpB riboswitch GFP pZa,pZa,bacterial,"Chloramphenicol, 25 μg/mL",37°C,DH5alpha,Low Copy,Green fluorescent protein,Bacteria,...,circular,source,Sequence,Full plasmid Sequence,[0:2871](+),TACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGC...,2871,None,None,0
97,170078,PclpB riboswitch GFP pZa,pZa,bacterial,"Chloramphenicol, 25 μg/mL",37°C,DH5alpha,Low Copy,Green fluorescent protein,Bacteria,...,circular,primer_bind,GFP-F,"GFP, forward primer. Does NOT anneal to EGFP",[434:455](+),GGTCCTTCTTGAGTTTGTAAC,21,None,None,0
98,170078,PclpB riboswitch GFP pZa,pZa,bacterial,"Chloramphenicol, 25 μg/mL",37°C,DH5alpha,Low Copy,Green fluorescent protein,Bacteria,...,circular,terminator,rrnB T1 terminator,transcription terminator T1 from the E. coli r...,[515:602](+),CAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTA...,87,None,None,0


In [64]:
# Keeping a reference dataframe as backup
full_df.to_csv("/scratch/alopatki_lab/Sharma/summer_project/db_blast_only_oriflag.csv")

In [12]:
for alignments in blast_res.alignments:
    print(alignments.title)
    for hsp in alignments.hsps:
        print(hsp.sbjct)

gb|UKB93153.1| TALEN [Transformation vector pJF1006]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
gb|UKB93145.1| TALEN [Transformation vector pJF1004]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
gb|UKB93149.1| TALEN [Transformation vector pJF1005]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
gb|UEZ83090.1| TAL effector nuclease [Cloning vector pTpTalen-R]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
YPYDVPDYAGSYPYDVPDYAAHGTVDL
pdb|6YVD|B Chain B, Condensin complex subunit 2 [Saccharomyces cerevisiae S288C]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
YPYDVPDYAGSYPYDVPDYAAGH
pdb|7QEN|C Chain C, Condensin complex subunit 2 [Saccharomyces cerevisiae] >pdb|7QFW|B Chain B, Condensin complex subunit 2 [Saccharomyces cerevisiae S288C]
YPYDVPDYAGYPYDVPDYAGSYPYDVPNYA
YPYDVPDYAGSYPYDVPNYAAGH
gb|AAT97346.1| GDBD-H4-Gcn5(F221A)-HA fusion protein [Yeast two-hybrid vector pDG4]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
gb|AAT97345.1| GDBD-H4-Gcn5-HA fusion protein [Yeast two-hybrid vector pDG3]
YPYDVPDYAGYPYDVPDYAGSYPYDVPDYA
gb|AAT97344.1| GDBD-H3-Gcn5(F221A)-HA fusion protein 

In [13]:
query_seq

'TACCCCTACGACGTGCCCGACTACGCCGGCTACCCATACGACGTCCCAGATTACGCTGGTTCGTACCCGTACGACGTGCCAGATTACGCC'

In [26]:
import pandas as pd
full_df = pd.read_csv("/scratch/alopatki_lab/Sharma/summer_project/db_wo_blast_uniprot.csv")


In [20]:
row = full_df.iloc[47]
seq = extract_cds(row)
res_1, res_2 = blast_search(seq)

In [24]:
res_1

['hypothetical',
 'conserved',
 'regulatory',
 'hypothetical',
 'regulatory',
 'hypothetical',
 'hypothetical',
 'MULTISPECIES:',
 'regulatory',
 'hypothetical']

In [28]:
full_df["Blast Title"] = None
full_df["Blast Sequence"] = None
# full["Uniprot"] = None
for i in range(len(full_df)):
    print(f"Currently processing row: {i+1}/{len(full_df)}")
    row = full_df.iloc[i]
    seq = extract_cds(row)
    if seq == -1:
        full_df.iloc[i]["Blast Title"] = None
        full_df.iloc[i]["Blast Sequence"] = None
    else:
        full_df.iloc[i]["Blast Title"], full_df.iloc[i]["Blast Sequence"] = blast_search(seq)
        print(full_df.iloc[i]["Blast Title"])
        print(full_df.iloc[i]["Blast Sequence"])
        
    clear_output(wait = True)

Currently processing row: 48/281556
None
None


/scratch/ashar58/tmp/ipykernel_116481/1503719945.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.iloc[i]["Blast Title"], full_df.iloc[i]["Blast Sequence"] = blast_search(seq)


NameError: name 'time' is not defined